# 파이썬으로 배우는 기계학습
# Machine Learning with Python
**************

# 제 8-3 강: 역전파 1

----------

## 학습 목표
- 역전파 개념의 이해 한다
- 다층 신경망에서 은닉층 오차를 계산하는 방법을 배운다.
- 은닉층 오차 계산을 단순화하고 일반화 하는 방법을 배운다.

## 학습 내용
- 출력층의 오차를 역전파하기
- 역전파로 은닉층의 오차 계산하기
- 은닉층 오차 계산의 단순화하고 일반화하기

------------

## 1. 여러 노드의 가중치 학습

페셉트론과 같은 단순 신경망을 이용한 선형 분류기에서는 예측값과 정답(클래스 레이블)과의 차이인 오차를 사용하여 학습하면서, 선형 분류자를 정교하게 조정할 수 있었습니다.  오차와 선형 분류자의 관계가 단순했기 때문에 기울기(가중치)를 조정해나가는 것이 어렵지 않았습니다.  

그런데 만약, 앞장에서 다룬 다음과 같은 신경망처럼 하나 이상의 노드가 결과를 출력하고, 따라서 하나 이상의 노드가 오차를 일으키는데 영향을 미친다면 가중치를 어떻게 조정해야 합리적일까요? 

<img src="images/2-3-2NN.png" width="400">
<center>그림 1: 신경망의 입력과 가중치</center>

## 2. 은닉층의 오차 계산
앞의 예제에서 클래스 레이블 $y_1 = 1$이고, 신경망 출력층의 결과 즉 예측값 $\hat{y_1} = 0.58$라고 합시다. 그러면, 오차($y_1 - \hat{y_1}$)는 0.42입니다. 이 오차를 활용하여 가중치를 조정하려고 할 때, 하나의 가중치에만 적용하는 것은 합리적이 못합니다. 왜냐하면, 3개 노드가 모두 오차를 발생하도록 영향을 끼쳤다고 보는 것이 타당하기 때문입니다.  그러므로 오차 0.42도 적절히 각 노드에게 분배하여 각 가중치를 조정하는 것이 합리적일 것입니다. 

그러면, 오차 0.42를 어떻게 합리적으로 분배해야 하나요? 이것이 우리의 질문입니다. 

<img src="images/2-3-2NN-BackProp1.png" width="500">
<center>그림 2: 신경망의 오차 역전파(1)</center>

균등하게 분배할 수도 있겠지만, 위의 예제에서 각각의 가중치 비중이 3:2:1이므로 오차도 같은 비례로 각 가중치에 나눠주는 것이 합당할 것입니다. 오차 0.42를 각각 $\frac{3}{6}:\frac{2}{6}:\frac{1}{6}$으로 배분해서 각 가중치를 조정할 때 고려하는 것입니다. 

이 방법은 노드가 더 많아질지라도 동일하게 적용할 수 있습니다. 만약 10개의 노드가 다음 층의 노드와 연결되어 있다면, 우리는 오차를 10개의 가중치에 나누어 주되, 그 값은 각각의 가중치의 크기에 따라 즉 오차에 영향을 끼친 값에 비례하여 분배하면 됩니다. 

여기서 잠깐 생각해보면, 가중치를 사용하는 데는 두 가지 다른 방법이 있다는 것을 관찰할 수 있습니다.  

- 첫째로 우리는 신경망의 입력층에서 출력층으로 신호를 전달할 때 가중치를 사용합니다. 지금까지 우리는 이렇게 순방향으로 입력신호를 전달할 때 사용하였습니다. 

- 둘째로 우리는 출력에서 다시 신경망으로 후진하여 오차를 반영하기 위해 가중치를 사용합니다. 이 방법을 역전파$^{backpropagation}$라 부르는 이유에 대해서는 별로 놀라지 않을 것입니다.

출력층에서의 오차 $E^{[2]}$는 출력층(2) 첫번째 노드이므로, 오차 $e^{[2]}_1$는 클래스 레이블$y_1$과 예측값$\hat{y}$과의 차이($e^{[2]}_1 = y_1 - \hat{y}$)입니다. 아랫첨자는 출력층의 노드가 여러 개인 것을 감안한 표기입니다.  우리가 나중에는 오차 제곱합(SSE: sum of squared error)을 사용하겠지만, 여기서는 일단 단순한 오차를 사용하기로 합시다.  그러면, 다음 그림과 같이 오차가 각 가중치의 값에 비례하여 배분을 할 수 있습니다. 

<img src="images/2-3-2NN-BackProp2.png" width="500">
<center>그림 3: 신경망의 오차 역전파 (2)</center>

<img src="images/2-3-2NN-BackProp3.png" width="500">
<center>그림 4: 신경망의 오차 역전파 (3)</center>

출력층의 첫 번째 노드에서의 오차 $e^{[2]}_1$와 두 번째 노드에서의 오차$e^{[2]}_2$를 계산하여 서로 합하여 은닉층과 출력층 사이에서 발생한 오차를 계산합니다.

\begin{align}
e_1^{[2]} &= y_1 - \hat{y}_1 \\  &= 1 - 0.58 = 0.42 \\
e_2^{[2]} &= y_2 - \hat{y}_2 \\  &= 1 - 0.55 = 0.45 \\
\\
E^{]2]} &= 
  \begin{pmatrix}  0.42 \\  0.45  \end{pmatrix}
\end{align}

결론적으로 출력층의 오차$E^{[2]}$를 은닉층과 출력층 사이의 각 가중치에 대하여 배분하여 은닉층의 오차$E^{[1]}$를 구하게 되었습니다.  여기서 $w_{ij}$는 모두 $W^{[2]}$행렬의 원소들입니다. 

\begin{align}
E^{[1]} &= 
  \begin{pmatrix}     e_1^{[1]} \\ e_2^{[1]} \\ e_3^{[1]} \\ \end{pmatrix} \\ &= 
  \begin{pmatrix}
     e^{[2]}_1 \cdot \frac{w_{11}}{\sum_{i=1}^{3} w_{i1}}  + 
     e^{[2]}_2 \cdot \frac{w_{12}}{\sum_{i=1}^{3} w_{i2}}  \\
     e^{[2]}_1 \cdot \frac{w_{21}}{\sum_{i=1}^{3} w_{i1}}   + 
      e^{[2]}_2 \cdot \frac{w_{22}}{\sum_{i=1}^{3} w_{i2}}   \\
     e^{[2]}_1 \cdot \frac{w_{31}}{\sum_{i=1}^{3} w_{i1}}   +
     e^{[2]}_2 \cdot \frac{w_{32}}{\sum_{i=1}^{3} w_{i2}}   \\
    \end{pmatrix} \\ &= 
    \begin{pmatrix}
    0.42 \cdot \frac{0.3}{0.3 + 0.2 + 0.1} +
    0.45 \cdot \frac{0.2}{0.2 + 0.1 + 0.1} \\
    0.42 \cdot \frac{0.2}{0.3 + 0.2 + 0.1} +
    0.45 \cdot \frac{0.1}{0.2 + 0.1 + 0.1} \\
    0.42 \cdot \frac{0.1}{0.3 + 0.2 + 0.1} +
    0.45 \cdot \frac{0.1}{0.2 + 0.1 + 0.1} \\
    \end{pmatrix} = 
    \begin{pmatrix} 0.21 + 0.23 \\ 0.14 + 0.11 \\ 0.07 + 0.11 \\ \end{pmatrix}  =
    \begin{pmatrix} 0.44 \\ 0.25 \\ 0.18 \\ \end{pmatrix}    \tag{1} 
\end{align}


$E^{[1]}$은 은닉층과 출력층 사이의 가중치로 인한 오차를 의미합니다.  간단히 은닉층의 오차라고 부릅니다.  출력층에 나타난 오차$E^{[2]}$을 가중치에 비례 배분하여 추정한 것이며, 다음과 같은 행렬 연산으로 정리가 됩니다.  

\begin{align}
E^{[1]} = 
  \begin{pmatrix}
    \frac{w_{11}}{\sum_{i=1}^{3} w_{i1}} & \frac{w_{12}}{\sum_{i=1}^{3} w_{i2}}  \\
    \frac{w_{21}}{\sum_{i=1}^{3} w_{i1}} & \frac{w_{22}}{\sum_{i=1}^{3} w_{i2}}  \\
    \frac{w_{31}}{\sum_{i=1}^{3} w_{i1}} & \frac{w_{32}}{\sum_{i=1}^{3} w_{i2}}  \\
  \end{pmatrix} 
  \cdot \begin{pmatrix} e_1^{[2]} \\ e_2^{[2]} \end{pmatrix} \tag{2} 
\end{align}

여기서 우리가 구한 $E^{[1]}$은 입력층과 은닉층 사이의 가중치 $W^{[1]}$를 조정할 때 한 부분으로 사용하게 될 것입니다.  물론 가중치 $W^{[2]}$를 조정하기 위하여 출력층의 오차 $E^{[2]}$를 공식의 일부로 사용합니다. 다음 강의는 바로 이 문제에 도전합니다. 

## 3. 오차 계산의 단순화와 일반화

그런데, $E^{[1]}$를 계산하는 식(2)의 왼쪽 행렬을 관찰해보면, 각 열의 분모가 모두 같은 것을 알 수 있습니다.  이것은 입력 자료들을 정규화는 하는 것과 같은 역할을 하는 것이므로 간소화할 수 있습니다.  따라서 분모 항목을 모두 제거해도 전체적인 학습 과정의 관점에서 가중치를 조정해 나가는 우리의 목적에는 아무런 영향을 미치지 않습니다.  왜냐하면, 신경망에서의 연산은 정확한 수치보다는 같은 방향(증가 혹은 감소)을 유지하면서 반복적으로 수행하는 특성때문입니다.  분모를 제거한 행렬을 관찰해보면, 이 행렬은 다음과 같이 간단해 집니다. 

\begin{align}  \tag{3}
E^{[1]} = 
\begin{pmatrix} w_{11} & w_{12}  \\ w_{21} & w_{22}  \\ w_{31} & w_{32}  \\  \end{pmatrix} 
\cdot \begin{pmatrix}  e_1^{[2]} \\ e_2^{[2]} \end{pmatrix} \end{align} 

식(3)의 가중치 행렬은 순전파에서 사용한 행렬과 비슷하지만 행과 열이 바뀌었습니다. 즉 전치가 되었습니다.  순전파에서 사용한 가중치 행렬 $W^{[2]}$를 전치해서 사용하면 된다는 것입니다. 그러므로, 식(3)은 다음과 같이 표현할 수 있습니다. 

\begin{align} E^{[1]} = W^{[2]T} \cdot E^{[2]} \tag{4} \end{align}  

결론적으로 은닉층의 오차 $E^{[1]}$는 은닉층의 가중치 $W^{[2]}$를 전치해서 출력층의 오차 $E^{[2]}$와 내적을 하여 구할 수 있습니다.   따라서 어떤 층의 오차 계산은 다음과 같이 출력으로부터 역추적하며 계산하는 방법으로 일반화할 수 있습니다. 

\begin{align} E^{[l]} = W^{[l+1]T} \cdot E^{[l+1]}  \tag{5} \end{align} 

아래와 같이 가중치가 주어지고, 초기값이 입력되었다고 합시다. 활성화 함수로는 시그모이드 함수를 사용하며, 클래스 레이블 $y$ 값도 아래와 같이 주어졌습니다. 예측값 $\hat{y}$을 구하고 그에 따른 오차를 계산해보겠습니다.

$
W^{(1)} = \left\lgroup \matrix{0.1 & 0.4 \cr 0.2 & 0.2 \cr 0.3 & 0.0} \right\rgroup,\space
W^{(2)} = \left\lgroup \matrix{0.3 & 0.2 & 0.1\cr 0.2 & 0.1 & 0.1} \right\rgroup,\space
\mathbf{x}^{(1)} = \left\lgroup \matrix{1 \cr 0} \right\rgroup,\space
y = \left\lgroup \matrix{1 \cr 1} \right\rgroup
$


In [1]:
import numpy as np
g = lambda x : 1 / (1 + np.exp(-x))
x = np.array([1, 0])

# feed forward propagation
W1 = np.array([[0.1, 0.4], [0.2, 0.2], [0.3, 0.0]])
z1 = np.dot(W1, x)
a1 = g(z1)
W2 = np.array([[0.3, 0.2, 0.1], [0.2, 0.1, 0.1]])
z2 = np.dot(W2, a1)
yhat = g(z2)
#print('yhat=', yhat)

# error backpropagation
y = np.array([1, 1])
E2 = y - yhat
E1 = np.dot(W2.T, E2)
#print('E1=', E1)

In [2]:
print("z1: {}".format(z1))
print("a1: {}".format(a1))
print("z2: {}".format(z2))
print("yhat: {}".format(yhat))
print("E1: {}".format(E1))
print("E2: {}".format(E2))

z1: [0.1 0.2 0.3]
a1: [0.52497919 0.549834   0.57444252]
z2: [0.32490481 0.21742349]
yhat: [0.58051912 0.55414275]
E1: [0.21501571 0.1284819  0.08653381]
E2: [0.41948088 0.44585725]


-------

## 학습 정리 
- 출력층으로부터 은닉층의 오차를 계산하기
- 복잡한 은닉층 오차 계산을 일반화 하기 

---------

## KMOOC 8-3 JoyQuiz 1 - Sigmoid

<img src="images/2-3-2NN.PNG" width="500">
<center>그림 4: 신경망의  순전파</center>

In [45]:
import numpy as np
g = lambda x : 1 / (1 + np.exp(-x))  # sigmoid
#g = lambda x : np.maximum(0, x)    # relu 
x = np.array([1, 0])

# feed forward propagation
W1 = np.array([[0.1, 0.4], [0.2, 0.2], [0.3, 0.0]])
z1 = np.dot(W1, x)
a1 = g(z1)
W2 = np.array([[0.5, 0.3, 0.4], [0.3, 0.2, 0.1]])
z2 = np.dot(W2, a1)
yhat = g(z2)
print('z1=', z1)
print('a1={}'.format(np.round(a1, 2)))
print('yhat={}'.format(np.round(yhat, 2)))

# error backpropagation
y = np.array([1, 1])
E2 = y - yhat
E1 = np.dot(W2.T, E2)
print("E2={}".format(np.round(E2, 2)))
print('E1={}'.format(np.round(E1, 2)))

z1= [0.1 0.2 0.3]
a1=[0.52 0.55 0.57]
yhat=[0.66 0.58]
E2=[0.34 0.42]
E1=[0.3  0.19 0.18]


## KMOOC 8-3 JoyQuiz 2, 3 - ReLU

<img src="images/2-3-2NNbackProp.PNG" width="500">
<center>그림 5: 신경망의 오차 역전파</center>

In [47]:
import numpy as np
#g = lambda x : 1 / (1 + np.exp(-x))  # sigmoid
g = lambda x : np.maximum(0, x)    # relu 
x = np.array([1, 0])

# feed forward propagatio
W1 = np.array([[0.1, 0.4], [0.2, 0.2], [0.3, 0.0]])
z1 = np.dot(W1, x)
a1 = g(z1)
W2 = np.array([[0.5, 0.3, 0.4], [0.3, 0.2, 0.1]])
z2 = np.dot(W2, a1)
yhat = g(z2)
print('z1=', z1)
print('a1={}'.format(np.round(a1, 2)))
print('yhat={}'.format(np.round(yhat, 2)))

# error backpropagation
y = np.array([1, 1])
E2 = y - yhat
E1 = np.dot(W2.T, E2)
print("E2={}".format(np.round(E2, 2)))
print('E1={}'.format(np.round(E1, 2)))

z1= [0.1 0.2 0.3]
a1=[0.1 0.2 0.3]
yhat=[0.23 0.1 ]
E2=[0.77 0.9 ]
E1=[0.66 0.41 0.4 ]


## KMOOC 8 HW 5 - 1, 2, 3 ReLU

<img src="images/2-3-2NN-Example.PNG" width="500">
<center>그림 6: 신경망의 순전파 (2)</center>

<img src="images/2-3-2NN-ExampleBackProp.PNG" width="500">
<center>그림 7: 신경망의 오차 역전파 (2)</center>

In [51]:
import numpy as np
#g = lambda x : 1 / (1 + np.exp(-x))  # sigmoid
g = lambda x : np.maximum(0, x)    # relu 
x = np.array([1, 0])

# feed forward propagation
W1 = np.array([[0.1, 0.4], [0.2, 0.2], [-0.3, 0.0]])
z1 = np.dot(W1, x)
a1 = g(z1)
W2 = np.array([[-0.3, 0.2, 0.1], [0.2, 0.1, 0.1]])
z2 = np.dot(W2, a1)
yhat = g(z2)
print('z1=', z1)
print('a1=', a1)
print('a2 = yhat=', yhat)

# error backpropagation
y = np.array([1, 1])
E2 = y - yhat
E1 = np.dot(W2.T, E2)
print("E2={}".format(np.round(E2, 2)))
print('E1={}'.format(np.round(E1, 2)))

z1= [ 0.1  0.2 -0.3]
a1= [0.1 0.2 0. ]
a2 = yhat= [0.01 0.04]
E2=[0.99 0.96]
E1=[-0.1   0.29  0.2 ]


----------
__JoyQuiz__: 앞의 강의에서 신경망 구조가 주어졌을 때에 신경망이 예측한 값을 계산하는 연습을 다양한 JoyQuiz 를 통해 살펴보았습니다. 렐루(ReLu) 활성화 함수를 사용하며, 아래와 같이 가중치를 주고, 입력값이 들어왔을 때, 신경망이 예측한 값 $\hat{y}$ 을 계산할 수 있습니다. 예측값과 실제값 (y) 와의 차이를 사용하여, 은닉층의 오차 ($E^{[1]}$) 와 출력층의 오차 ($E^{[2]}$) 를 계산하세요. 아래의 보기에서 오차값을 선택하세요.

$
W^{(1)} = \left\lgroup \matrix{0.1 & 0.9 \cr 0.5 & 0.7 \cr 0.3 & 0.7} \right\rgroup,\space
W^{(2)} = \left\lgroup \matrix{0.1 & 0.9 & 0.3\cr 0.5 & 0.7 & 0.2} \right\rgroup,\space
\mathbf{x}^{(1)} = \left\lgroup \matrix{1 \cr 0} \right\rgroup,\space
y = \left\lgroup \matrix{1 \cr 1} \right\rgroup
$

Q1: $E^{[1]}$ 을 선택하세요

(1) $E^{[1]} = \left\lgroup \matrix{0.315 \cr 0.783 \cr 0.243} \right\rgroup \space$
(2) $E^{[1]} = \left\lgroup \matrix{0.215 \cr 0.555 \cr 0.123} \right\rgroup \space$
(3) $E^{[1]} = \left\lgroup \matrix{0.015 \cr 0.255 \cr 0.123} \right\rgroup \space$
(4) $E^{[1]} = \left\lgroup \matrix{0.215 \cr 0.155 \cr 0.523} \right\rgroup \space$

__JoyAnswer__: (1)

Q2: $E^{[2]}$  을 선택하세요

(1) $E^{[2]} = \left\lgroup \matrix{0.15 \cr 0.55} \right\rgroup \space$
(2) $E^{[2]} = \left\lgroup \matrix{0.45 \cr 0.54} \right\rgroup \space$
(3) $E^{[2]} = \left\lgroup \matrix{0.25 \cr 0.35} \right\rgroup \space$
(4) $E^{[2]} = \left\lgroup \matrix{0.35 \cr 0.95} \right\rgroup \space$

__JoyAnswer__: (2)

In [3]:
#g = np.vectorize(lambda x : x if x > 0 else 0) # wrong
g = lambda x: np.maximum(0, x)
x = np.array([1, 0])

# feed forward propagation
W1 = np.array([[0.1, 0.9], [0.5, 0.7], [0.3, 0.7]])
z1 = np.dot(W1, x)
a1 = g(z1)
W2 = np.array([[0.1, 0.9, 0.3], [0.5, 0.7, 0.2]])
z2 = np.dot(W2, a1)
yhat = g(z2)

# error backpropagation
y = np.array([1, 1])
E2 = y - yhat
E1 = np.dot(W2.T, E2)

In [4]:
print("z1: {}".format(z1))
print("a1: {}".format(a1))
print("z2: {}".format(z2))
print("yhat: {}".format(yhat))
print("E1: {}".format(E1))
print("E2: {}".format(E2))

z1: [ 0.1  0.5  0.3]
a1: [ 0.1  0.5  0.3]
z2: [ 0.55  0.46]
yhat: [ 0.55  0.46]
E1: [ 0.315  0.783  0.243]
E2: [ 0.45  0.54]


----------
__JoyQuiz__: 앞의 강의에서 신경망 구조가 주어졌을 때에 신경망이 예측한 값을 계산하는 연습을 다양한 JoyQuiz 를 통해 살펴보았습니다. 렐루(ReLu) 활성화 함수를 사용하며, 아래와 같이 가중치를 주고, 입력값이 들어왔을 때, 신경망이 예측한 값 $\hat{y}$ 을 계산할 수 있습니다. 예측값과 실제값 (y) 와의 차이를 사용하여, 은닉층의 오차 ($E^{[1]}$) 와 출력층의 오차 ($E^{[2]}$) 를 계산하세요. 아래의 보기에서 오차값을 선택하세요.

$
W^{(1)} = \left\lgroup \matrix{0.1 & -0.9 \cr -0.5 & 0.7 \cr -0.3 & 0.7} \right\rgroup,\space
W^{(2)} = \left\lgroup \matrix{-0.1 & 0.9 & -0.3\cr -0.5 & 0.7 & -0.2} \right\rgroup,\space
\mathbf{x}^{(1)} = \left\lgroup \matrix{1 \cr 0} \right\rgroup,\space
y = \left\lgroup \matrix{1 \cr 1} \right\rgroup
$

Q1: $E^{[1]}$ 을 선택하세요

(1) $E^{[1]} = \left\lgroup \matrix{0.3 \cr 0.7 \cr 0.24} \right\rgroup \space$
(2) $E^{[1]} = \left\lgroup \matrix{0.15 \cr 0.5 \cr 0.13} \right\rgroup \space$
(3) $E^{[1]} = \left\lgroup \matrix{0.5 \cr 0.25 \cr 0.23} \right\rgroup \space$
(4) $E^{[1]} = \left\lgroup \matrix{-0.6 \cr 1.6 \cr -0.5} \right\rgroup \space$

__JoyAnswer__: (4)

Q2: $E^{[2]}$  을 선택하세요

(1) $E^{[2]} = \left\lgroup \matrix{0 \cr 0} \right\rgroup \space$
(2) $E^{[2]} = \left\lgroup \matrix{0 \cr 1} \right\rgroup \space$
(3) $E^{[2]} = \left\lgroup \matrix{1 \cr 0} \right\rgroup \space$
(4) $E^{[2]} = \left\lgroup \matrix{1 \cr 1} \right\rgroup \space$

__JoyAnswer__: (4)

In [5]:
#g = np.vectorize(lambda x : x if x > 0 else 0) # wrong
g = lambda x: np.maximum(0, x)
x = np.array([1, 0])
W1 = np.array([[0.1, -0.9], [-0.5, 0.7], [-0.3, 0.7]])
z1 = np.dot(W1, x)
a1 = g(z1)
W2 = np.array([[-0.1, 0.9, -0.3], [-0.5, 0.7, -0.2]])
z2 = np.dot(W2, a1)
yhat = g(z2)

y = np.array([1, 1])
E2 = y - yhat
E1 = np.dot(W2.T, E2)

In [6]:
print("z1: {}".format(z1))
print("a1: {}".format(a1))
print("z2: {}".format(z2))
print("yhat: {}".format(yhat))
print("E1: {}".format(E1))
print("E2: {}".format(E2))

z1: [ 0.1 -0.5 -0.3]
a1: [ 0.1  0.   0. ]
z2: [-0.01 -0.05]
yhat: [0 0]
E1: [-0.6  1.6 -0.5]
E2: [1 1]


----------
__JoyQuiz__: 앞의 강의에서 신경망 구조가 주어졌을 때에 신경망이 예측한 값을 계산하는 연습을 다양한 JoyQuiz 를 통해 살펴보았습니다. 이번에는 은닉층에서 5개의 뉴런을 사용합니다. 렐루(ReLu) 활성화 함수를 사용하며, 아래와 같이 가중치를 주고, 입력값이 들어왔을 때, 신경망이 예측한 값 $\hat{y}$ 을 계산할 수 있습니다. 예측값과 실제값 (y) 와의 차이를 사용하여, 은닉층의 오차 ($E^{[1]}$) 와 출력층의 오차 ($E^{[2]}$) 를 계산하세요. 아래의 보기에서 오차값을 선택하세요.

$
W^{(1)} = \left\lgroup \matrix{0.1 & -0.9 \cr -0.5 & 0.7 \cr -0.3 & 0.7 \cr 0.1 & -0.9 \cr -0.5 & 0.7} \right\rgroup,\space
W^{(2)} = \left\lgroup \matrix{-0.1 & 0.9 & -0.3 & 0.7 & -0.2\cr -0.5 & 0.7 & -0.2 & 0.9 & -0.3} \right\rgroup,\space
\mathbf{x}^{(1)} = \left\lgroup \matrix{1 \cr 0} \right\rgroup,\space
y = \left\lgroup \matrix{1 \cr 1} \right\rgroup
$

Q1: $E^{[1]}$ 을 선택하세요

(1) $E^{[1]} = \left\lgroup \matrix{0.315 \cr 0.783 \cr 0.243 \cr 0.476\cr 1.522} \right\rgroup \space$
(2) $E^{[1]} = \left\lgroup \matrix{-0.574 \cr 1.518 \cr -0.474 \cr 1.522\cr -0.476} \right\rgroup \space$
(3) $E^{[1]} = \left\lgroup \matrix{-0.315 \cr 1.522 \cr -0.474 \cr 1.315\cr -0.783} \right\rgroup \space$
(4) $E^{[1]} = \left\lgroup \matrix{-0.574 \cr 1.476 \cr -0.315 \cr 1.783\cr -0.476} \right\rgroup \space$

__JoyAnswer__: (2)

Q2: $E^{[2]}$  을 선택하세요

(1) $E^{[2]} = \left\lgroup \matrix{0.94 \cr 0.96} \right\rgroup \space$
(2) $E^{[2]} = \left\lgroup \matrix{0.96 \cr 0.54} \right\rgroup \space$
(3) $E^{[2]} = \left\lgroup \matrix{0.25 \cr 0.94} \right\rgroup \space$
(4) $E^{[2]} = \left\lgroup \matrix{0.35 \cr 0.95} \right\rgroup \space$

__JoyAnswer__: (1)

In [7]:
#g = np.vectorize(lambda x : x if x > 0 else 0) # wrong
g = lambda x: np.maximum(0, x)
x = np.array([1, 0])
W1 = np.array([[0.1, -0.9], [-0.5, 0.7], [-0.3, 0.7], [0.1, -0.9], [-0.5, 0.7]])
z1 = np.dot(W1, x)
a1 = g(z1)
W2 = np.array([[-0.1, 0.9, -0.3, 0.7, -0.2], [-0.5, 0.7, -0.2, 0.9, -0.3]])
z2 = np.dot(W2, a1)
yhat = g(z2)

y = np.array([1, 1])
E2 = y - yhat
E1 = np.dot(W2.T, E2)

In [8]:
print("z1: {}".format(z1))
print("a1: {}".format(a1))
print("z2: {}".format(z2))
print("yhat: {}".format(yhat))
print("E1: {}".format(E1))
print("E2: {}".format(E2))

z1: [ 0.1 -0.5 -0.3  0.1 -0.5]
a1: [ 0.1  0.   0.   0.1  0. ]
z2: [ 0.06  0.04]
yhat: [ 0.06  0.04]
E1: [-0.574  1.518 -0.474  1.522 -0.476]
E2: [ 0.94  0.96]


__JoyQuiz__: 조금 더 복잡한 신경망을 구현해봅니다. 은닉층이 2개 있으며, 각각의 은닉층이 3개의 뉴런을 사용하며, 시그모이드 활성화 함수를 사용하는 신경망이 있다고 합시다. 아래와 같이 가중치를 주고, 입력값이 들어왔을 때, 신경망이 예측한 값 $\hat{y}$ 을 계산할 수 있습니다. 예측값과 실제값 (y) 와의 차이를 사용하여, 은닉층의 오차 ($E^{[1]}, E^{[2]}$) 와 출력층의 오차 ($E^{[3]}$) 를 계산하세요. 아래의 보기에서 오차값을 선택하세요.

$
W^{(1)} = \left\lgroup \matrix{0.1 & -0.9 \cr -0.5 & 0.7 \cr -0.3 & 0.7} \right\rgroup,\space
W^{(2)} = \left\lgroup \matrix{0.4 & 0.5 & -0.2 \cr -0.1 & 0.9 & -0.3\cr -0.5 & 0.7 & -0.2} \right\rgroup,\space
W^{(3)} = \left\lgroup \matrix{-0.1 & 0.9 & -0.3\cr -0.5 & 0.7 & -0.2} \right\rgroup,\space
\mathbf{x}^{(1)} = \left\lgroup \matrix{1 \cr 0} \right\rgroup,\space
y = \left\lgroup \matrix{1 \cr 1} \right\rgroup
$

Q1: $E^{[1]}$ 을 선택하세요

(1) $E^{[1]} = \left\lgroup \matrix{-0.323 \cr 0.712 \cr 0.241} \right\rgroup \space$
(2) $E^{[1]} = \left\lgroup \matrix{0.151 \cr -0.546 \cr 0.135} \right\rgroup \space$
(3) $E^{[1]} = \left\lgroup \matrix{-0.077 \cr 0.357 \cr -0.117} \right\rgroup \space$
(4) $E^{[1]} = \left\lgroup \matrix{-0.631 \cr 1.623 \cr -0.564} \right\rgroup \space$

__JoyAnswer__: (3)

Q2: $E^{[2]}$  을 선택하세요

(1) $E^{[2]} = \left\lgroup \matrix{-0.294 \cr 0.738 \cr -0.229} \right\rgroup \space$
(2) $E^{[2]} = \left\lgroup \matrix{0.229 \cr -0.738 \cr -1.294} \right\rgroup \space$
(3) $E^{[2]} = \left\lgroup \matrix{-0.294 \cr 0.229 \cr -0.738} \right\rgroup \space$
(4) $E^{[2]} = \left\lgroup \matrix{0.294 \cr 0.738 \cr 1.229} \right\rgroup \space$

__JoyAnswer__: (1)

Q2: $E^{[3]}$  을 선택하세요

(1) $E^{[3]} = \left\lgroup \matrix{-0.430 \cr 0.502} \right\rgroup \space$
(2) $E^{[3]} = \left\lgroup \matrix{-0.502 \cr 0.430} \right\rgroup \space$
(3) $E^{[3]} = \left\lgroup \matrix{0.502 \cr 0.430} \right\rgroup \space$
(4) $E^{[3]} = \left\lgroup \matrix{0.430 \cr 0.502} \right\rgroup \space$

__JoyAnswer__: (4)



In [9]:
g = lambda x : 1 / (1 + np.exp(-x))
x = np.array([1, 0])
W1 = np.array([[0.1, -0.9], [-0.5, 0.7], [-0.3, 0.7]])
z1 = np.dot(W1, x)
a1 = g(z1)

W2 = np.array([[0.4, 0.5, -0.2], [-0.1, 0.9, -0.3], [-0.5, 0.7, -0.2]])
z2 = np.dot(W2, a1)
a2 = g(z2)

W3 = np.array([[-0.1, 0.9, -0.3], [-0.5, 0.7, -0.2]])
z3 = np.dot(W3, a2)
yhat = g(z3)

y = np.array([1, 1])
E3 = y - yhat
E2 = np.dot(W3.T, E3)
E1 = np.dot(W2.T, E2)

In [10]:
print("z1: {}".format(z1))
print("a1: {}".format(a1))
print("z2: {}".format(z2))
print("yhat: {}".format(yhat))
print("E1: {}".format(E1))
print("E2: {}".format(E2))
print("E3: {}".format(E3))

z1: [ 0.1 -0.5 -0.3]
a1: [ 0.52497919  0.37754067  0.42555748]
z2: [ 0.31365051  0.15962144 -0.08332262]
yhat: [ 0.57060176  0.49828758]
E1: [-0.07670315  0.35658001 -0.11670554]
E2: [-0.29379603  0.73765711 -0.22916196]
E3: [ 0.42939824  0.50171242]
